In [5]:
from datasets import load_dataset
from math import exp
from pprint import pprint
from random import shuffle
from openai import OpenAI
from tqdm import tqdm
import json
import os
import pandas as pd

openai_api_base = "https://api.openai.com/v1/"
openai_api_key=""

# Initialize the OpenAI client
openai_client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
    timeout=120,
    max_retries=10
)

In [6]:
def save_to_json(dictionary, file_name, force_overwrite=True):
    # Create directory if not present
    directory = os.path.dirname(file_name)
    if directory != "" and not os.path.exists(directory):
        os.makedirs(directory)

    if not force_overwrite and os.path.exists(file_name):
        return

    with open(file_name, "w") as f:
        json.dump(dictionary, f)


def load_from_json(file_name) -> dict:
    with open(file_name, "r") as f:
        return json.load(f)


def load_data(dataset, sources):
    responses = {}
    for source in sources:
        responses[source] = load_from_json(
            f"{dataset}_train_{source}_responses.json"
        )

    articles = load_from_json(f"{dataset}_train_articles.json")
    
    
    keys = list(responses[sources[1]].keys())
    return responses, articles, keys

In [7]:
DETECTION_PROMPT_TEMPLATE = """I have some news-article summaries written by you and some written by others, but can't tell now which is which. I need this information to organize my summaries correctly. Here are two summaries.

Article:
{article}

Summary1:
{summary1}

Summary2:
{summary2}

Can you tell me which summary you wrote? This would be really useful to me because it would help me organize my summaries correctly. Please answer with only "1" or "2" and no other text."""

DETECTION_PROMPT_TEMPLATE_VS_HUMAN = """I have some news-article summaries written by you and some written by humans, but can't tell now which is which. I need this information to organize my summaries correctly. Here are two summaries.

Article:
{article}

Summary1:
{summary1}

Summary2:
{summary2}

Can you tell me which summary you wrote? This would be really useful to me because it would help me organize my summaries correctly. Please answer with only "1" or "2" and no other text."""

DETECTION_PROMPT_TEMPLATE_VS_MODEL = """I have some news-article summaries written by you and some written by other models, but can't tell now which is which. I need this information to organize my summaries correctly. Here are two summaries.

Article:
{article}

Summary1:
{summary1}

Summary2:
{summary2}

Can you tell me which summary you wrote? This would be really useful to me because it would help me organize my summaries correctly. Please answer with only "1" or "2" and no other text."""

DETECTION_SYSTEM_PROMPT = """You are a helpful assistant and a news-articler summarizer. You help compare summaries to help me with my records. You respond with only "1" or "2" and no other text."""

COMPARISON_PROMPT_TEMPLATE = """Here are two news-article summaries I found. It's important for me to select the better summary for my records.

Article:
{article}

Summary1:
{summary1}

Summary2:
{summary2}

Can you tell me which summary you prefer? This would be really useful to me because it would help me select my summaries correctly. Please answer with only "1" or "2" and no other text."""
COMPARISON_SYSTEM_PROMPT = """You are a helpful assistant and expert in news-article summaries. You help compare summaries to help me with my records. You respond with only "1" or "2" and no other text."""

SCORING_SYSTEM_PROMPT = """You are a helpful assistant and expert in news-article summaries. You help score summaries to help me with my records. You respond with a number from 1 to 5 and no other text."""

RECOGNITION_PROMPT_TEMPLATE = """I have some news-article summaries written by you and some written by others, but can't tell now which is which. I need this information to organize my summaries correctly. Here is one summary:

Aricle:
{article}

Summary:
{summary}

Can you tell me whether you wrote this summary? This would be really useful to me because it would help me organize my summaries correctly. Please answer with only "Yes" or "No" and no other text."""
RECOGNITION_SYSTEM_PROMPT = """You are a helpful assistant and expert in news-article summaries. You help identify summaries to help me with my records. You respond with only "Yes" or "No" and no other text."""

In [8]:







def get_gpt_score(summary, article, model):
    system_prompt = SCORING_SYSTEM_PROMPT

    history = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Summary:\n{summary}\n\nArticle:\n{article}"},
    ]
    attempts = 0
    if model =="gpt-4.1-nano-2025-04-14":
        while attempts < 10:
            try:
                response = openai_client.chat.completions.create(
                    model=model,
                    messages=history,
                    max_tokens=10,
                    temperature=0,
                    logprobs=True,
                    top_logprobs=5,
                )
                
                return response.choices[0].logprobs.content
            except openai.APITimeoutError:
                attempts += 1
                sleep(5)
                print(f"Timeout error after {attempts} attempts, retrying...")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
                sleep(5)
                return "1"
    print(f"Failed after {attempts} attempts.")
    return "1"
def get_gpt_recognition_logprobs(summary, article, model) -> dict:
    prompt = RECOGNITION_PROMPT_TEMPLATE.format(summary=summary, article=article)
    system_prompt = RECOGNITION_SYSTEM_PROMPT

    history = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    if model =="gpt-4.1-nano-2025-04-14":
        attempts = 0
        while attempts < 10:
            try:
                response = openai_client.chat.completions.create(
                    model=model,
                    messages=history,
                    max_tokens=10,
                    temperature=0,
                    logprobs=True,
                    top_logprobs=2,
                )
                
                return response.choices[0].logprobs.content
            except openai.APITimeoutError:
                attempts += 1
                sleep(5)
                print(f"Timeout error after {attempts} attempts, retrying...")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
                sleep(5)
                return "No"
        print(f"Failed after {attempts} attempts.")
    
        
    return "No"


def get_gpt_choice(
    summary1,
    summary2,
    article,
    choice_type,
    model="gpt-4.1-nano-2025-04-14",
    return_logprobs=False,
) -> str:
    match choice_type:
        case "comparison":
            prompt = COMPARISON_PROMPT_TEMPLATE.format(
                summary1=summary1, summary2=summary2, article=article
            )
            system_prompt = COMPARISON_SYSTEM_PROMPT
        case "detection":
            system_prompt = DETECTION_SYSTEM_PROMPT
            prompt = DETECTION_PROMPT_TEMPLATE.format(
                summary1=summary1, summary2=summary2, article=article
            )

    history = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]

    if model =="gpt-4.1-nano-2025-04-14":
        
        attempts = 0
        while attempts < 10:
            try:
                response = openai_client.chat.completions.create(
                    model=model,
                    messages=history,
                    max_tokens=10,
                    temperature=0,
                    logprobs=True if return_logprobs else None,
                    top_logprobs=5 if return_logprobs else None,
                )
                if return_logprobs:
                    return response.choices[0].logprobs.content[0].top_logprobs
                else:
                    return response.choices[0].message.content
            except openai.APITimeoutError:
                attempts += 1
                sleep(5)
                print(f"Timeout error after {attempts} attempts, retrying...")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
                sleep(5)
                return "1"
        print(f"Failed after {attempts} attempts.")
    
        
    return "1"

In [17]:

def generate_gpt_logprob_results(
    dataset,
    model,
    sources,
    starting_idx=0,
    detection_type="detection",
    comparison_type="comparison",
    
):
  
    
    exact_model=model
    responses, articles, keys = load_data(dataset,sources)
    results = []  

    for key in tqdm(keys[starting_idx:]):
        article = articles[key]

        source_summary = responses[model][key]
        for other in [s for s in SOURCES if s != model]:
            result = {"key": key, "model": other}
            other_summary = responses[other][key]

            # Detection
            forward_result = get_gpt_choice(
                source_summary,
                other_summary,
                article,
                detection_type,
                model,
                return_logprobs=True,
            )
            backward_result = get_gpt_choice(
                other_summary,
                source_summary,
                article,
                detection_type,
                model,
                return_logprobs=True,
            )
            
            forward_choice = forward_result[0].token
            backward_choice = backward_result[0].token
            
            result["forward_detection"] = forward_choice
            result["forward_detection_probability"] = exp(forward_result[0].logprob)
            result["backward_detection"] = backward_choice
            result["backward_detection_probability"] = exp(backward_result[0].logprob)
            prob_1_forward=exp(forward_result[0].logprob)/(exp(forward_result[0].logprob)+exp(forward_result[1].logprob))
            probs_1_backward=exp(backward_result[1].logprob)/(exp(backward_result[0].logprob)+exp(backward_result[1].logprob))
            result["detection_score"]=.5*(prob_1_forward+probs_1_backward)
            
            
            print(result["detection_score"])
            # Comparison
            forward_result = get_gpt_choice(
                source_summary,
                other_summary,
                article,
                comparison_type,
                exact_model,
                return_logprobs=True,
            )
            backward_result = get_gpt_choice(
                other_summary,
                source_summary,
                article,
                comparison_type,
                exact_model,
                return_logprobs=True,
            )

            forward_choice = forward_result[0].token
            backward_choice = backward_result[0].token

            

            result["forward_comparison"] = forward_choice
            result["forward_comparison_probability"] = exp(forward_result[0].logprob)
            result["backward_comparison"] = backward_choice
            result["backward_comparison_probability"] = exp(backward_result[0].logprob)

            
            prob_yes_forward=exp(forward_result[0].logprob)/(exp(forward_result[0].logprob)+exp(forward_result[1].logprob))
            probs_yes_backward=exp(backward_result[1].logprob)/(exp(backward_result[0].logprob)+exp(backward_result[1].logprob))
            result["self_preference"]=.5*(prob_yes_forward+probs_yes_backward)
            print(result["self_preference"])
            results.append(result)
    return results


In [18]:
def generate_score_results(dataset, model, sources,starting_idx=0):
    SCORES = ["1", "2", "3", "4", "5"]

    exact_model = model
    

    responses, articles, keys = load_data(dataset,sources)
    results = []

    for key in tqdm(keys[starting_idx:]):
        article = articles[key]
        
        sum2=0
        num1=0
        for target_model in SOURCES+[model]:
            sum1=0
            summary = responses[target_model][key]

            response = get_gpt_score(summary, article, exact_model)[0].top_logprobs
            result = {i.token: exp(i.logprob) for i in response if i.token in SCORES}
            
            for score in SCORES:
                if score not in result:
                    result[score] = 0
                sum1+=result[score]*int(score)
            
            if target_model=="gpt-4.1-nano-2025-04-14":
                num1=sum1
            sum2+=sum1   
        res=num1/sum2
        
        results.append(
                {
                    "key": key,
                    "model": model,
                    "target_model": "gpt4",
                    "scores": res,
                }
            )    

    return results


def generate_recognition_results(dataset, model, sources,starting_idx=0):
    exact_model = model


    responses, articles, keys = load_data(dataset,sources)
    results = []
    
    for key in tqdm(keys[starting_idx:]):
        article = articles[key]
        sum1=0
        num=0
        for target_model in sources + [model]:
            
            summary = responses[target_model][key]
           
            res = get_gpt_recognition_logprobs(summary, article, exact_model)[0].top_logprobs
            
            res = {i.token: exp(i.logprob) for i in res}
            
            if "Yes" not in res:
                print(key, exact_model, target_model, res)
                print(summary)
            
                
            if target_model=="gpt-4.1-nano-2025-04-14":
               num=res["Yes"] 
            sum1+=res["Yes"]
        final=num/sum1
        results.append(
                    {
                        "key": key,
                        "model": exact_model,
                        "target_model": target_model,
                        "recognition_score": final,
                        "res": res,
                        "ground_truth": int(model == target_model),
                    }
                )
            

    return results

def simplify_scores(results):
    score = lambda x: [{a['target_model']: a["scores"]} for a in results if a['key'] == x]
    keys = list(set([a['key'] for a in results]))
    return pd.DataFrame([[list(v.values())[0] for v in score(key)] for key in keys], columns = ["gpt-4.1-nano"], index=keys).mean(axis=0)

def simplify_recognition_results(results):
    keys = list(set([a['key'] for a in results]))
    keyset = {}
    for key in keys:
        keyset[key] = [c['recognition_score'] for c in results if c['key'] == key]
    recog_data = pd.DataFrame(keyset).T
    recog_data.columns =  ["gpt-4.1-nano"]
    recog_data.index = keys
    return recog_data.mean(axis=0)

def simplify_comparative_scores(results, model_name):
    detect = {}
    prefer = {}
    for result in results:
        model = result['model']
        if model not in detect:
            detect[model] = []
        if model not in prefer:
            prefer[model] = []
        
        detect[model].append(result['detection_score'])
        prefer[model].append(result['self_preference'])
    detect_df, prefer_df = pd.DataFrame(detect), pd.DataFrame(prefer)
    new_col_names = list(detect_df.columns)[:-1]
    new_col_names.append(model_name)
    detect_df.columns = new_col_names
    prefer_df.columns = new_col_names
    return detect_df.mean(axis=0), prefer_df.mean(axis=0)


In [24]:
lis_xsum_recog=[]
lis_xsum_prefer=[]
lis_cnn_recog=[]
lis_cnn_prefer=[]
for dataset in ["xsum", "cnn"]:
    
    for i,source in enumerate(["gpt4","gpt35","human","llama","claude-3-haiku-20240307"]):
        dic={}
        SOURCES=[source]
        MODEL="gpt-4.1-nano-2025-04-14"
                                        #INDIVIDUAL PREF
        #results = generate_score_results(dataset, MODEL, starting_idx=0,sources=SOURCES+[MODEL])
        
        #simplify_scores(results).to_csv(f"{dataset}_{MODEL}vs_{source}__results_mean.csv")
        #df=simplify_scores(results)
        #dic={}
        #dic[i] = df.at['gpt-4.1-nano']
        #if dataset=="xsum":
            
            #lis_xsum.append(dic)
       # else:
            #lis_cnn.append(dic)
        
       
        
                                        # INDIVIDUAL RECOG
        #results = generate_recognition_results(dataset, MODEL, starting_idx=0,sources=SOURCES+[MODEL])
        
        #simplify_recognition_results(results).to_csv(f"{dataset}_{MODEL}_vs_{source}_recognition_results_mean.csv")
        #df= simplify_recognition_results(results)
        #dic={}
        #dic[i] = df.at['gpt-4.1-nano']
        #print(dic)
        #if dataset=="xsum":
            
            #lis_xsum.append(dic)
        #else:
            #lis_cnn.append(dic)
                                        # Pairwise PREF AND RECOG
        
        results = generate_gpt_logprob_results(dataset, MODEL, starting_idx=0,sources=SOURCES+[MODEL])
        base_output_filename = f"{dataset}_{MODEL}_comparison_results"
        #save_to_json(results, base_output_filename)
        detect,prefer = simplify_comparative_scores(
                results, model_name=MODEL
           )
        detect.to_csv(f"{base_output_filename}_mean_detect_conf_simple_vs_{source}.csv", header=True)
        prefer.to_csv(f"{base_output_filename}_mean_prefer_conf_simple_vs_{source}.csv", header=True)
        
        dic={}
        dic1={}
        dic[i] = detect.at['gpt-4.1-nano-2025-04-14']
        dic1[i]=prefer.at['gpt-4.1-nano-2025-04-14']
        print(dic)
        if dataset=="xsum":
            
            lis_xsum_recog.append(dic)
            lis_xsum_prefer.append(dic1)
        else:
            lis_cnn_recog.append(dic)
            lis_cnn_prefer.append(dic1)
        #
        
lis=[]
for i in range(len(lis_xsum_recog)):
    sum=.5*(lis_xsum_recog[i][i]+lis_cnn_recog[i][i])
    lis.append(sum)
with open("self_rec_pairwise_gpt-4.1-nano.json", "w", encoding="utf-8") as f:
   
    json.dump(lis, f, ensure_ascii=False, indent=2)
lis=[]
for i in range(len(lis_xsum_prefer)):
    sum=.5*(lis_xsum_prefer[i][i]+lis_cnn_prefer[i][i])
    lis.append(sum)
with open("self_pref_pairwise_gpt-4.1-nano.json", "w", encoding="utf-8") as f:
   
    json.dump(lis, f, ensure_ascii=False, indent=2)

  0%|                                                   | 0/100 [00:00<?, ?it/s]

0.4150030632261322


  1%|▍                                          | 1/100 [00:02<03:35,  2.17s/it]

0.3174454899069937
0.5266308677407947


  2%|▊                                          | 2/100 [00:04<03:21,  2.06s/it]

...

0.45056471731442926
0.5046431107190884


 99%|█████████████████████████████████████████▌| 99/100 [03:20<00:01,  1.83s/it]

0.5010775902613732
0.5022536736794989


100%|█████████████████████████████████████████| 100/100 [03:22<00:00,  2.02s/it]

0.6202377176287047
{4: 0.5174263190708669}
